# Creating maps using OSM data

Before we get started, we need to import the necessary libraries.

In [ ]:
import osmnx as ox
from IPython.display import Image

%matplotlib inline
ox.config(log_console=True)
ox.__version__

## Plot Aachen street network type
OpenStreetMap's data are useful to study urban form, networks, resilience and accessibility. [Street network](https://geoffboeing.com/2017/04/urban-form-analysis-openstreetmap/) data especially when holding these cities at the same scale providing spatial objectivity in comparing urban forms. Comparing the following city networks (all cities a part of the rheinisch revier) show essential differences and outline, the differences and their uniqueness ![Street_Network](./img/streetNetwork.png).  

For the first plot we need to define the dpi, the size (make sure it is not too big, or it may take some time to be plotted) and of course a place. These are essential inputs for the plot_figure_ground method. But the map is not just visualized as the output here but also saved as **aachen.png**. If parameters are unclear or unknown, have a look at the documentation, every parameter is described there.

In [ ]:
dpi=72
size=600
place = "Aachen, Germany"
fp = f"./aachen.png"
fig, ax = ox.plot_figure_ground(
    address=place,
    network_type="walk",
    filepath=fp,
    dpi=dpi,
    save=True,
    show=False,
    close=True,
)
Image(fp, height=size, width=size)

## Let us create a figure ground plan
Next step, creating a figure ground plan of Aachen. Therefore, we first have to set a centre point, a distance and a tag. The function creates a GeoDataFrame of OSM with a distance to N, S, W, E. The short form gdf stands for GeoDataFrame.

In [ ]:
tags={"building":True}
point = (50.775593, 6.082478)
size=500
dist = 1000
gdf = ox.geometries_from_point(point, tags, dist=dist)
gdf_proj = ox.project_gdf(gdf)


In [ ]:
gdf.head()

The output shows, that in the column geometry polygons show up. The function plot_footprints takes these polygons to create the footprints of the buildings. The figure-ground diagrams are essential when representing the relationship between the build and unbuild space. The probably most known example is the **Nolli Map** of Rome.

In [ ]:
bbox = ox.utils_geo.bbox_from_point(point=point, dist=dist, project_utm=True)
fp = f"./aachen_schwarzplan.png"
fig, ax = ox.plot_footprints(
    gdf_proj,
    bbox=bbox,
    bgcolor="white",
    color="black",
    filepath=fp,
    dpi=300,
    save=True,
    show=False,
    close=True,
)
Image(fp, height=size, width=size)

### Exercise
Create a figure ground plan for your hometown or a city by choice, choose a suitable size.

## Analysing information 
Information as GeoDataFrame are Next we do slice the GeoDataFrame for extracting information. Let us print the Table again and decide for what t

In [ ]:
gdf.head()

In [ ]:
gdf['amenity'].unique()

In [ ]:
gdf[gdf ["amenity"] == "cafe"].head()

In [ ]:
gdf['building'].unique()

In [ ]:
gdf[gdf['building'] == "university"].plot()

In [ ]:
reiff = ox.geocode("reiff museum aachen")
hbf = ox.geocode("Aachen Hauptbahnhof")


In [ ]:
rdf = ox.geometries_from_point(reiff, tags={"building":True},dist=100)
rdf.head()


In [ ]:
fig = rdf.plot()

## Let us work with graphs
Spatial graphs can be traceable to a mathematical concept. A mathematical graph is composed of **nodes** which are linked by **edges**. Therefore, nodes can represent geographic places with specific locations and edges represent connecting paths. The composition is needed to analyse structures, such as the following example of path calculation shows.

In [ ]:
# attention, getting the whole of Aachen takes many minutes, we will get the inner city only
#G = ox.graph_from_place("Aachen, Germany", network_type="walk")
G = ox.graph_from_point(point, dist=500, network_type="bike")


In [ ]:
ax, fig = ox.plot_graph(G, node_color='b', node_zorder=3, bgcolor='#FFFFFF')

In the following we are going to plot the shortest path between a start and an end point. There is one issue the actual train station is not part of the map itself, so the closest point to the train station is chosen. If you pick two nodes, then the shortest path between those two can be found, using weight = "length". One-way streets are correctly handled. 

In [ ]:
start = ox.nearest_nodes(G, reiff[1], reiff[0])
dest  = ox.nearest_nodes(G, hbf[1], hbf[0])

The shortes_path() function comes from the NetworX library. If the weight parameter of this function is specified the Dijkstra's algorithm is used to find the optimal route. This algorithm belongs to the class of Greedy algorithms. In graph theory these are used to solve optimiztion problems such as finding the shortest path. 

In [ ]:
route = ox.shortest_path(G, start, dest)
ox.plot_graph_route(G, route)

## Saving graphs
Sometimes one need this picture printed as PNG or JPG or a more geo related file such as GeoPackage.

In [ ]:
ox.save_graph_geopackage(G, filepath="mynetwork.gpkg")

Other times you may need your graphs to be able to load in for example illustrator and therefore saved as **svg** file. When running the code below, a new file will show up in your file structure, called mynetwork.svg.

In [ ]:
ox.plot_graph_route(G, route, node_color='b', node_zorder=3, bgcolor='#FFFFFF', show=False, save=True, filepath='./mynetwork.svg')

### Exercise
Add another path, for example from the Aachen university hospital to the Aachen city hall. You are welcome to choose another path, but the map must be extended and for practise do not reuse Reiff and Hbf. Print both the existing and the new path in one map use different colours. Hint: Checkout the OSMnx documentation for functions such as plot_graph_route().  